# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

В ходе исследования постараемся выяснить как ключевые показатели, указанные в таблице влияют на своевременность погашения кредита заемщиком.

На первом этапе познакомимся с таблицей, посмотрим какие показатели имеются, с чем предстоит работать. Далее подготовим таблицу для анализа: избавимся от пропусков, дубликатов, приведем таблицу к комфортному для анализа виду. Проведем изучение ключевых характеристик клиентов в разрезе своевременного возврата заемных средств.

## Обзор данных

Составим первое представление о данных 

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение его в переменную data
data = pd.read_csv('/datasets/data.csv')

In [3]:
# получение первых 10 строк таблицы data 
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице всего 12 колонок, 5 из которых принадлежат типу `object`, 5 типу `int64` и еще 2 типу `float64`

Согласно документации к данным:
* `children`     — количество детей в семье;
* `days_employe` — общий трудовой стаж в днях;  
* `dob_years`    — возраст клиента в годах;
* `education`    — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender`       — пол клиента;
* `income_type`  — тип занятости;  
* `debt`         — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose`      — цель получения кредита;

Каждая строка таблицы - сведения о заемщике. В колонках описываются данные помогающие составить представление о социальном статусе клиента: количество детей, трудовой стаж, уровень образования, семейное положение, тип занятости, ежемесячный доход, возраст, пол. Остальные данные рассказывают непосредственно о кредитной истории клиента: имел ли задолженность по возврату кредита, цель получения кредита.

Содержание колонок, а также количество строк является достаточным для проведения анализа и ответа на поставленные вопросы.
При этом в некоторых колонках встречаются пропуски. Перед тем как перейти к анализу необходимо устранить пропуски, проверить есть ли другие проблемы с данными, при наличии устранить. 

##  Заполнение пропусков

Пропуски в данных появляются разными путями. Например, пользователь не указал о себе какие-нибудь сведения или автоматизированная система сбора информации дала сбой. Иногда пропуски оставляют умышленно, рассчитывая на автозаполнение спецсимволами.

Посмотрим какова доля пропущеных значений в колонках 'days_employed' и 'total_income'. Мы не должны удалять более 5-7% данных, чтобы коренным образом не повлиять на конечный результат анализа.

In [5]:
data.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Доля пропущенных значений равна 0.1, тогда удалив строки с None, мы лишим себя почти 1/10 данных. Это повлияет на выводы исследования. Тем не менее, пропуски нельзя оставлять без внимания, поэтому заполним обнаруженные пропуски в колонке 'total_income'.

Пропущенные значения можно заполнить средним значением или медианой. Если тех, кто получает высокую зарплату, столько же, сколько тех, кто получает небольшие деньги, — среднее подойдёт. Но когда есть оторванные от основной массы лидеры, их результаты сильно смещают значение среднего вверх. Вот почему оценивать данные широкого круга людей лучше медианой.

In [6]:
data['total_income'] = data['total_income'].fillna(value = data['total_income'].median()) # в аргумет метода fillna() подставим
                                                                                    # медианное значение колонки total_income

In [7]:
data.info() # проверим заполнились ли пропуски

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Проверка данных на аномалии и исправления

При знакомстве с данными колонки 'days_employed' обнаружены отрицательные значения. Это можно воспринять как ошибку вызванную человеческим фактором, поэтому избавимся от знака минус во всех этих числах.

In [8]:
data['days_employed'] = data['days_employed'].abs() # избавимся от знака минус в колонке 

Данные колонки days_employed взаимодействуют с колонкой dob_years. Общий трудовой стаж должен корректно соотноситься с возрастом человека. Официальный возраст, с которого можно работать согласно заканодательству равен 16 лет. Соответственно трудовой стаж который будет превышать разницу между текущем возрастом и шестнадцатью лет, будет являться аномальным и не подходит нам для анализа. Избавимся от этих значений, заменив их на '0'.

In [9]:
rows = data['days_employed'] > (data['dob_years'] * 365 - 16 * 365) # вычислим максимально возможное количество дней стажа
                                                                    # сравним с имеющимся количество. Все, что больше-аномалия

data.loc[rows, 'days_employed'] = '0' # сначала заменяем на '0', чтобы отсечь аномалии при поиске медианного значения

data.head(10)   # проверим результат замены 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.42261,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


У оставшихся данных колонки days_employed определим медианное значение. Подставим его вместо пропусков, а также вместо аномальных значений.

In [10]:
rows = data['days_employed'] == '0' # создаем список с необходимым значением для логической индексации


days_employed_median = data['days_employed'].median() # вычисляем медианное значение колонки и сохраняем в переменную
data.loc[rows, 'days_employed'] = days_employed_median # с помощью логической индексации заменяем все значения "0" на медиан.зн.
data['days_employed'] = data['days_employed'].fillna(value = days_employed_median) # заменяем пропуски на медианное значение

data.info() # проверяем остались ли пропуски                         

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Изменение типов данных

Для удобства заменим значения колонки с данными об общем трудовом стаже в днях и ежемесячном доходе с вещественного значения на целочисленный

In [11]:
data['total_income'] = data['total_income'].astype('int32')

In [12]:
data['days_employed'] = data['days_employed'].astype('int32')

Замена типа данных с int64 на int32 позволит съэкономить память

In [13]:
data['children'] = data['children'].astype('int32')

In [14]:
data['dob_years'] = data['dob_years'].astype('int32')

In [15]:
data['education_id'] = data['education_id'].astype('int32')

In [16]:
data['family_status_id'] = data['family_status_id'].astype('int32')

In [17]:
data['debt'] = data['debt'].astype('int32')

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int32 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int32 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int32 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int32 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int32 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(7), object(5)
memory usage: 1.4+ MB


Использование память уменьшилось с 2.0+ MB до 1.4+ MD. 

## Удаление дубликатов

В колонке одно и то же значение может быть записано по разному (разный регистр, язык, опечатки) — это неявные дубликаты. Используем метод unique(). Он возвращает перечень уникальных значений в столбце

In [19]:
data['education'].unique() 

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Среди уникальных значений ищут неявные дубликаты. Например ВЫСШЕЕ, Высшее и высшее. Неправильные и альтернативные написания значений исправляют методом replace(). 

In [20]:
data['education'] = data['education'].str.lower()

In [21]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [22]:
data['education'].unique() 

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Проверим оставшиеся колонки на наличие неявных дубликатов.

*******


In [23]:
data['dob_years'].unique() 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int32)

Среди уникальных значений имеется 0. Так, как возраст клиента, взявшего кредит не может быть равен 0, очевидно, что данное значение не подходит для анализа. Выявим у скольких человек в графе возраст указано 0.

In [24]:
incorrect_1 = len(data.loc[data['dob_years'] == 0]) # посчитаем количество строк со значением 0 в колонке и сохраним в перемен.

In [25]:
data_total = len(data) # посчитаем общее количество строк в датафрейме

Определим на сколько данное количество существенно для анализа.

In [26]:
incorrect_total_1 = incorrect_1/data_total
print(f'{incorrect_total_1:.1%}')

0.5%


Количество строк меньше одного процента, значит мы можешь удалить данный строки, не боясь, что их отсутствие определяющим образом скажется на результате.

In [27]:
data = data.loc[data['dob_years'] != 0] # удаляем все строки со значением 0 в колонке, перезаписав датафрейм

Убедимся, что значение 0 удалено.

In [28]:
data['dob_years'].unique() 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int32)

*****

Выведем уникальные значения колонки 'children'

In [29]:
data['children'].unique() 

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int32)

Значения -1 и 20 являются аномальнымии не подходят для анализа 

In [30]:
incorrect_2 = len(data.loc[data['children'] == -1]) + len(data.loc[data['children'] == 20])

In [31]:
incorrect_total_2 = incorrect_2/data_total
print(f'{incorrect_total_2:.1%}')

0.6%


Данное количество строк не критично для анализа, поэтому удалим их из таблицы.

In [32]:
data = data.loc[(data['children'] != -1) & (data['children'] != 20)] # используем логическую индексацию, чтобы перезаписать 
#датафрейм, исключив строки с неподходящими значениями 

Проверим, что мы избавились от этих значений

In [33]:
data.groupby('children').count() 

,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
children,,,,,,,,,,,
0,14080,14080,14080,14080,14080,14080,14080,14080,14080,14080,14080
1,4802,4802,4802,4802,4802,4802,4802,4802,4802,4802,4802
2,2042,2042,2042,2042,2042,2042,2042,2042,2042,2042,2042
3,328,328,328,328,328,328,328,328,328,328,328
4,41,41,41,41,41,41,41,41,41,41,41
5,9,9,9,9,9,9,9,9,9,9,9


******


Выведем на экран уникальные значения столбца 'family_status'

In [34]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Здесь все показатели представлены корректно

***

Аналогичная ситуация со столбцом 'education_id'. Показатели не вызываю вопросов.

In [35]:
data['education_id'].unique() 

array([0, 1, 2, 3, 4], dtype=int32)

******


Посмотрим уникальные знаения в следующей колонке

In [36]:
data['gender'].unique() 

array(['F', 'M', 'XNA'], dtype=object)

Показатель 'XNA' некорректный и не подходит для анализа. Выясним количество строк с данным значением, чтобы понять, насколько их удаление повлияет на результат анализа.

In [37]:
data.groupby('gender').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,income_type,debt,total_income,purpose
gender,,,,,,,,,,,
F,14083,14083,14083,14083,14083,14083,14083,14083,14083,14083,14083
M,7218,7218,7218,7218,7218,7218,7218,7218,7218,7218,7218
XNA,1,1,1,1,1,1,1,1,1,1,1


В таблице присутствует всего одна строка с данным показателем, поэтому мы можем ее удалить.

In [38]:
data = data.loc[data['gender'] != 'XNA']

Проверим результат

In [39]:
data.groupby('gender').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,income_type,debt,total_income,purpose
gender,,,,,,,,,,,
F,14083,14083,14083,14083,14083,14083,14083,14083,14083,14083,14083
M,7218,7218,7218,7218,7218,7218,7218,7218,7218,7218,7218


*****


Выведем на экран уникальные значения столбца 'income_type'. Здесь все значения представлены корректно

In [40]:
data['income_type'].unique() 

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

*******


Аналогичная ситуация со столбцом 'debt'. Показатели не вызываю вопросов.

In [41]:
data['debt'].unique() 

array([0, 1], dtype=int32)

***


Метод duplicated() ищет дубликаты. По умолчанию он признаёт дубликатами те строки, которые полностью повторяют уже встречавшиеся в датасете. Основные причины возникновения дубликатов — повторные представления, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации. В первую очередь выясним количество явных дубликатови удалим их.

In [42]:
data.duplicated().sum() # вычисляем количество явных дубликатов

71

In [43]:
data = data.drop_duplicates().reset_index(drop=True) # удалим дубликаты и заново проиндексируем датафрейм

data.duplicated().sum() # проверим остались ли какие-нибудь явные дубликаты

0

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создадим новый DataFrame на основании data. Каждому уникальному значению из 'education' соответствует уникальное значение 'education_id'

In [44]:
data_education = data[['education','education_id']]

In [45]:
data_education # познакомимся с таблицей

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21225,среднее,1
21226,среднее,1
21227,среднее,1
21228,среднее,1


**********


Создадим еще один новый DataFrame на основании data. Каждому уникальному значению из 'family_status' соответствует уникальное значение 'family_status_id'

In [46]:
data_family_status = data[['family_status','family_status_id']]

In [47]:
data_family_status

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
...,...,...
21225,гражданский брак,1
21226,женат / замужем,0
21227,гражданский брак,1
21228,женат / замужем,0


*******

Из датафрейма data удалим колонки 'education' и 'family_status'

In [48]:
data = data.drop(columns = ['education', 'family_status'], axis = 1) # axis = 1 - удаляем столбцы

## Категоризация дохода

Создадим функцию, которая будет присваивать каждой строчке категорию доходности на основании заработка клиента

In [49]:
def income_category(total_income):
    """
    Возвращает категорию дохода по значению дохода total_income, используя правила:
    - 0–30000 — 'E';
    - 30001–50000 — 'D';
    - 50001–200000 — 'C';
    - 200001–1000000 — 'B';
    - 1000001 и выше — 'A'.
    """
    
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A' 

Добавим соответствующую колонку в датафрейм data

In [50]:
data['total_income_category'] = data['total_income'].apply(income_category) #применим функцию к столбцу total_income
# результаты сохраним в новый столбец

Проверим результат выведя на экран первый пять строк таблицы.

In [51]:
data.head() # ознакомимся с результатами

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,1177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


## Категоризация целей кредита

Создайте функцию, которая на основании данных из столбца 'purpose' сформирует новый столбец 'purpose_category', в который войдут следующие категории:

- 'операции с автомобилем',

- 'операции с недвижимостью',

- 'проведение свадьбы',

- 'получение образования'.

In [52]:
def purpose_category(purpose):
    """
    Возвращает категорию целей кредита согласно цели , используя правила:
    """
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образовани' in purpose:
        return 'получение образования'
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    return 'операции с недвижимостью' 

Добавим соответствующую колонку в датафрейм data

In [53]:
data['purpose_category'] = data['purpose'].apply(purpose_category)

Проверим результат, выведя на экран первые двадцать строк таблицы

In [54]:
data.head(20)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,1177,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


## Ответы на вопросы

### Вопрос 1

Проведем анализ показателей, которые могут влиять на возврат кредита в срок. 

Создадим сводную таблицу влияния количества детей на возврат кредита в срок 

In [55]:
data_pivot_1 = data.pivot_table(columns='children', values='debt',aggfunc='mean') 
#columns — столбец, по значениям которого происходит группировка 
#values — значения, по которым мы хотим увидеть сводную таблицу 
#aggfunc — функция, применяемая к значениям (медиана) 
data_pivot_1

children,0,1,2,3,4,5
debt,0.075458,0.092028,0.095145,0.082317,0.097561,0.0


****


Теперь с помощью сводной таблицы определим зависимость между семейным положением и задолженностями по кредиту.

In [56]:
data_pivot_2 = data.pivot_table(columns=data_family_status['family_status'], values='debt',aggfunc='mean')
data_pivot_2


family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
debt,0.097842,0.071247,0.065539,0.093142,0.075575


****

Создадим сводную таблицу по уровню дохода и возвратом кредита в срок

In [57]:
data_pivot_3 = data.pivot_table(columns='total_income_category', values='debt',aggfunc='mean')
data_pivot_3

total_income_category,A,B,C,D,E
debt,0.08,0.070798,0.084921,0.060519,0.090909


****

Создадим сводную таблицу влияния целей кредита на возврат долга в срок.

In [58]:
data_pivot_4 = data.pivot_table(columns='purpose_category', values='debt',aggfunc='mean')
data_pivot_4

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,0.093236,0.072596,0.092947,0.07873


### Вывод 1

Сгруппировав значения по различным показателям можно сделать следующие выводы. 

Отсутствие детей у заемщиков повышает вероятность возврата кредита в срок. Наличие детей может нести риск возникновения незапланированных трат. Не имея детей, о которых необходимо заботиться деньги можно тратить только на себя, можно лучше спланировать бюджет.

Люди находящиеся или находившиеся в браке являются более дисциплинированными заемщиками. Это может быть связано с более ответственным подходом к жизни, умением вести бюджет, планировать траты. 

Нельзя отметить серьезную зависимость между доходом и фактом возврата кредита в срок, за исключением ситуации, когда у клиента совсем низкий уровень зароботка. В таком случае могут возникать проблемы. Денег в таком случае может не хватать, что негативно сказывается на своевременный возврат кредита.

Как показывает анализ кредиты, взятые на операции с недвижимостью и проведение свадьбы возвращают более дисциплинированно, чем кредиты, взятые на операции с автомобилем и получение образования. К Можно сделать вывод, что к покупке недвижимости и обслуживанию кредита, взятого на эти цели клиенты подходят более ответственно, как к необходимому имуществу. Образование и автомобиль не являются жизненнонеобходимыми приобретениями и это может откладывать соответствующий отпечатам на дисциплину возврата кредита.

## Общий вывод

От банка была получена таблица платежеспособности клиента. На первом этапе изучили основную информацию о таблице. 
Затем приступили к предобработке данных. Избавились от аномальных показателей, а пропущенные заменили медианным значением.
На следующем этапе избавились от явных и неявных дубликатов, чтобы избавиться от показателей, которые могут негативно повлиять на анализ.

Для удобства изменили тип данных float на int, где это показалось целесообразным, создали новые датафреймы и перемистили туда некоторые колонки, чтобы сделать основной датафрейм более удобным для анализа.

Провели категоризацию доходов и целей кредита, чтобы систематизировать информацию для более удобного и качественного анализа данных показателей. Полученные данные добавили в основной датафрейм.

Для анализа мы использовали метод, строящий сводные таблицы. Построение сводных таблиц позволяет быстро анализировать данные и сразу подготовить отчёты.

Проанализировав таблицу, можно сделать выводы, что идеальный заемщик для банка - это семейный человек, не имеющий детей, получающий доход выше необходимого минимума (A-D), основной целью которого является получения кредита для приобретения недвижимости или проведения свадьбы